In [1]:
import numpy as np 
import pandas as pd
import geopandas as gpd

In [2]:
# Load shape file
# Downloaded from https://www.census.gov/cgi-bin/geo/shapefiles/index.php
block_shapefile = gpd.read_file("tl_2023_39_tabblock20/tl_2023_39_tabblock20.shp")

In [3]:
# Load crime data to filter the census blocks 
crime_data = pd.read_csv('crime_incidents.csv')

C:\Users\sgdea\AppData\Local\Temp\ipykernel_18136\2773976917.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_data = pd.read_csv('crime_incidents.csv')


In [4]:
# Find the unique census blocks in the crime data 
crime_census_blocks = np.unique(crime_data['CENSUS_BLOCK_GEOID'])

# Filter the shape data to include only these blocks 
block_shapefile_filtered = block_shapefile[block_shapefile['GEOID20'].isin(crime_census_blocks)]

# Check to make sure that all of the blocks matched 
set(crime_census_blocks)  - set(block_shapefile_filtered['GEOID20'])

{'Not Located'}

In [5]:
# Calculate the center of each block 

# Ensure the GeoDataFrame is in a projected CRS before calculating centroids
block_shapefile_filtered  = block_shapefile_filtered.to_crs(epsg=3857)

# Calculate the centroid of the block group geometry
block_shapefile_filtered ['centroid'] = block_shapefile_filtered .geometry.centroid

# Convert centroid back to latitude and longitude
centroid_lat_lon = block_shapefile_filtered ['centroid'].to_crs(epsg=4326)  # Convert back to WGS84

# Assign latitude and longitude to new columns
block_shapefile_filtered ['lat_center'] = centroid_lat_lon.y
block_shapefile_filtered ['lon_center'] = centroid_lat_lon.x

In [6]:
# Create a separate dataframe for the blocks 
census_block_loc = block_shapefile_filtered[['GEOID20', 'lat_center', 'lon_center']].reset_index(drop=True)
census_block_loc.columns.values[0] = 'geo_id'

In [8]:
# Save the csv to be using for mapping other attributes 
census_block_loc.to_csv('census-block-locations.csv')

In [9]:
len(census_block_loc)

6671